##### Using autoencoder, we're gonna make embedding vectors for each building_id.
I expect the autoencoder can retract some informative feature via unsupervised learning.<br>
For example, the similar pattern between buildings could be repersented as numeric values.<br>
If we know this values, the final model will be more robust since he won't need to learn that values from the final learning step.

#### Load modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as K

from sklearn.preprocessing import StandardScaler

#### Load dataset

In [2]:
df = pd.read_pickle('train.pickle')
df.dropna(axis='columns', inplace=True)

building_id = df.building_id.values.reshape(-1, 1)
others = StandardScaler().fit_transform(df.drop(columns='building_id'))

In [3]:
building_id.shape

(19869988, 1)

In [4]:
others.shape

(19869988, 18)

#### Build autoencoder

In [5]:
def construct():
    K.clear_session()
    
    building_id = keras.layers.Input(shape=1)
    building_embedding = keras.layers.Embedding(1449, 32)(building_id)
    building_embedding = keras.layers.Flatten()(building_embedding)
    
    others = keras.layers.Input(shape=18)
    prev = keras.layers.Concatenate()([building_embedding, others])
        
    prev = keras.layers.Dense(16)(prev)
    prev = keras.layers.BatchNormalization(momentum=0.8)(prev)
    prev = keras.layers.Activation('elu')(prev)
    
    prev = keras.layers.Dense(8)(prev)
    prev = keras.layers.BatchNormalization(momentum=0.8)(prev)
    prev = keras.layers.Activation('elu')(prev)
    
    prev = keras.layers.Dense(16)(prev)
    prev = keras.layers.BatchNormalization(momentum=0.8)(prev)
    prev = keras.layers.Activation('elu')(prev)
    
    prev = keras.layers.Dense(18)(prev)
    
    autoencoder = keras.Model(inputs=[building_id, others], outputs=prev)
    embedding = keras.Model(inputs=building_id, outputs=building_embedding)
    
    return autoencoder, embedding
    
autoencoder, embedding = construct()

In [6]:
early_stopping = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

autoencoder.compile(loss='mse')
autoencoder.fit(
    x=[building_id, others],
    y=others,
    batch_size=65536,
    epochs=100,
    callbacks=[early_stopping],
    validation_split=0.2
)

Train on 15895990 samples, validate on 3973998 samples
Epoch 1/100
15895990/15895990 [==============================] - 10s 1us/sample - loss: 0.4662 - val_loss: 0.2105
Epoch 2/100
15895990/15895990 [==============================] - 7s 0us/sample - loss: 0.1669 - val_loss: 0.1529
Epoch 3/100
15895990/15895990 [==============================] - 7s 0us/sample - loss: 0.1246 - val_loss: 0.1242
Epoch 4/100
15895990/15895990 [==============================] - 7s 0us/sample - loss: 0.1036 - val_loss: 0.1068
Epoch 5/100
15895990/15895990 [==============================] - 7s 0us/sample - loss: 0.0927 - val_loss: 0.1008
Epoch 6/100
15895990/15895990 [==============================] - 7s 0us/sample - loss: 0.0886 - val_loss: 0.0982
Epoch 7/100
15895990/15895990 [==============================] - 7s 0us/sample - loss: 0.0866 - val_loss: 0.0963
Epoch 8/100
15895990/15895990 [==============================] - 7s 0us/sample - loss: 0.0852 - val_loss: 0.0953
Epoch 9/100
15895990/15895990 [=========

#### Save trained embedding model

In [7]:
embedding.save('building')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: building\assets
